# Домашнее задание 2


---



In [ ]:
import numpy as np

Простая ф-ция для проверки решения на примерах указанных в задании

In [ ]:
def run_test(method, test):
  test_input, ans = test
  t_type, a, b, eps1, eps2, x_0, x_1 = test_input
  f, df = (f1, [df1_1, df1_2]) if t_type == 0 else (f2, [df2_1, df2_2])
  x = [x_0, x_1]
  res = method(f, df,a,b,x,10000,0.5, eps1, eps2)
  print('CALCULATED RES:', res)
  assert(abs(res - ans) < 0.1)
  print('TEST PASSED!')

In [ ]:
def run_test_H(method, test):
  test_input, ans = test
  a, b, eps1, eps2, x_0, x_1 = test_input
  df = [df_1,df_2]
  H = [[df_1_1,df_1_2],[df_2_1,df_2_2]]
  x = [x_0, x_1]
  res = method(f, df, H, a, b, x, 10000, 0.5, eps1, eps2)
  print('CALCULATED RES:', res)
  assert(abs(res - ans) < 0.1)
  print('TEST PASSED!')

In [ ]:
def fib_search(f, bounds, tol, coef, max_eps = 0.01):
    n = 0
    fibN = 1
    fibNm1 = 0
    fibNm2 = 0
    l = min(bounds)
    r = max(bounds)
    interv_l = (r - l) / tol
    while fibN < interv_l:
      fibNm2 = fibNm1
      fibNm1 = fibN
      fibN = fibNm1 + fibNm2
      n += 1
    p1 = l + fibNm2/fibN * (r - l)
    p2 = l + fibNm1/fibN * (r - l)
    fp1 = f(p1, *coef)
    fp2 = f(p2, *coef)
    for k in range(1, n-1):
      fibN = fibNm1
      fibNm1 = fibNm2
      fibNm2 = fibN - fibNm1
      if (fp1 > fp2):
        l = p1
        p1 = p2
        p2 = l + (fibNm1/fibN) * (r - l)
        fp1 = fp2
        fp2 = f(p2, *coef) 
      else:
        r = p2
        p2 = p1
        p1 = l + (fibNm2/fibN) * (r - l)
        fp2 = fp1
        fp1 = f(p1, *coef)
    p2 = l + (1/2 - max_eps) * (r - l)
    if fp1 >= f(p2, *coef):
      r = p1
    elif fp1 < f(p2, *coef):
      l = p2
    return (r+l)/2

# A. Метод покоординатного спуска



---



В данной задаче Вам необходимо реализовать метод Покоординатного спуска для функций двух типов:

$f_0(x) = a \sin(x_0) + b \cos(x_1)$

$f_1(x) = (x_0 - a)^2 - x_0 x_1 + (x_1 - b)^2$

\

Максимальная длина шага - 0.5

Критерии остановки:
  * Длина градиента в точке меньше $\epsilon_1$

  * Количество итераций меньше 10000 (гарантируется)

  * Две итерации подряд $||x^{k+1} - x^{k}|| < \epsilon_2$ и $||f(x^{k+1}) - f(x^{k})|| < \epsilon_2$ 

  
Можете воспользоваться [шаблоном](https://gist.github.com/evkonovalov/04d212db54ddbfef650f796d8965c1f4).

\

**Формат ввода:**

$t$ - тип функции (0 или 1)

$a, b$ - коэффициенты

$\epsilon_1, \epsilon_2$ - критерии остановки

$x_0, x_1$ - координаты начальной точки

\

**Формат вывода:**

Значение целевой функции в точке. Проверяется до первого знака после запятой.

In [ ]:
import numpy as np

def f1(x,a,b):
    return a*np.math.sin(x[0]) + b*np.math.cos(x[1])

def f2(x,a,b):
    return (x[0] - a)**2 + x[0]*x[1] + (x[1] - b)**2

def df1_1(x,a,b):
    return a*np.math.cos(x[0])

def df1_2(x,a,b):
    return -b*np.math.sin(x[1])

def df2_1(x,a,b):
    return -2*a+2*x[0]+x[1]

def df2_2(x,a,b):
    return -2*b+x[0]+2*x[1]

def coordinate_descent(f, df, a,b,x00, M, t0, eps1, eps2):
  eps = 0.5
  def eps2_check(x_j0, x_j1, f_vals=None):
    if not f_vals:
      f_vals = [f(x_j0, a, b), f(x_j1, a, b)]
    return (np.linalg.norm(x_j1 - x_j0) < eps2 and np.linalg.norm(f_vals[1] - f_vals[0]) < eps2)
  
  def coord_step(x_prev, coord, t_i):
    grad = np.array([df[0](x_prev, a, b), df[1](x_prev, a, b)])
    x_new = x_prev - t_i * grad * coord
    f_value = f(x_new, a, b)
    dif = f_value - f_prev
    if (dif >= 0) and (dif >= -eps * np.linalg.norm(grad)):
      t_i /= 2
      return False, t_i, x_new, f_value
    return True, t_i, x_new, f_value

  x_prev = x00
  f_prev = f(x_prev, a, b)
  t = np.array([t0, t0])
  e = np.eye(2)
  prev_step_flag = False
  f_k, x_k = 0, 0
  j = 0
  while True:
    cond, t[j % 2], x_k, f_k = coord_step(x_prev, e[j % 2], t[j % 2])
    flag = eps2_check(x_prev, x_k, [f_prev, f_k])
    if (flag and prev_step_flag) or (j >= 2 * M):
      return min(f_k, f_prev)
    prev_step_flag = flag
    if (not cond):
      continue
    x_prev = x_k
    f_prev = f_k
    j += 1
  return min(f_k, f_prev)

In [ ]:
type = int(input())
a,b = map(float,input().split(" "))
if type == 0:
    f = f1
    df = [df1_1,df1_2]
else:
    f = f2
    df = [df2_1,df2_2]
eps1,eps2 = map(float,input().split(" "))
x1,x2 = map(float,input().split(" "))
x = [x1,x2]
print(coordinate_descent(f,df,a,b,x,10000,0.5,eps1,eps2))

In [ ]:
test1 = [(0, -2, -2, 0.0001, 0.0006, 0.7, 0.9), -4.0]
test2 = [(1, 2, 1,0.001, 0.0001, 0.8, 0.1), 1.000000114440918]

In [ ]:
run_test(coordinate_descent, test1)
run_test(coordinate_descent, test2)

CALCULATED RES: -4.0
TEST PASSED!
CALCULATED RES: 1.0000000004470349
TEST PASSED!


# B. Метод сопряжённых градиентов


---

\

В данной задаче Вам необходимо реализовать метод сопряженных градиентов для функций двух типов:

$f_0(x) = (x_0 - a)^2 + x_1^2 + \frac{x_0}{|x_1| + b}$

$f_1(x) = (x_0 - a)^2 + x_0x_1 + (|x_1| + b)^2$

\

Максимальная длина шага - 0.5. 
В тестовом решении, для поиска оптимального длины шага, использовался поиск Фибоначчи. Вы можете использовать другие методы.
Критерии остановки:
  * Длина градиента в точке меньше $\epsilon_1$

  * Количество итераций меньше 10000 (гарантируется)

  * Две итерации подряд $||x^{k+1} - x^{k}|| < \epsilon_2$ и $||f(x^{k+1}) - f(x^{k})|| < \epsilon_2$ 

Можете воспользоваться [шаблоном](https://gist.github.com/evkonovalov/b643fc8a5cbf28c812e408c0bebe0739)

\

**Формат ввода:**

$t$ - тип функции (0 или 1)

$a, b$ - коэффициенты

$\epsilon_1, \epsilon_2$ - критерии остановки

$x_0, x_1$ - координаты начальной точки

\

**Формат вывода:**

Значение целевой функции в точке. Проверяется до первого знака после запятой.

In [ ]:
import numpy as np

def f1(x,a,b):
    return (x[0]-a)**2 + x[1]**2 + x[0]/(np.math.fabs(x[1])+b)

def f2(x,a,b):
    return (x[0] - a)**2 + x[0]*x[1] + (x[1] - b)**2

def df1_1(x,a,b):
    return -2*a+1/(np.math.fabs(x[1])+b)+2*x[0]

def df1_2(x,a,b):
    return 2*x[1]-(x[0]*x[1])/(np.math.fabs(x[1])*(np.math.fabs(x[1])+b)**2)

def df2_1(x,a,b):
    return -2*a+2*x[0]+x[1]

def df2_2(x,a,b):
    return -2*b+x[0]+2*x[1]

def fletcher_reeves(f, df, a, b, x0, M, t0, eps1, eps2):
    def get_grad(x_k):
        return np.array([df[0](x_k, a, b), df[1](x_k, a, b)])
      
    def calc_beta(grad_xk, grad_prev):
        return np.linalg.norm(grad_xk)/np.linalg.norm(grad_prev) if grad_prev is not None else 0
    
    def calc_dk(grad_xk, beta, d_prev=None):
        return -grad_xk + beta * d_prev if (d_prev is not None) else -grad_xk

    def calc_tk(x_k, d_k):
        def f_wrapped(t, x_k, d_k, a, b):
          return f(x_k + t * d_k, a ,b)
        return fib_search(f_wrapped, bounds=[0, 0.5], tol=0.001, coef=[x_k, d_k, a, b])
    
    def eps2_check(x_j0, x_j1, f_vals=None):
        if not f_vals:
          f_vals = [f(x_j0, a, b), f(x_j1, a, b)]
        return (np.linalg.norm(x_j1 - x_j0) < eps2 and np.linalg.norm(f_vals[1] - f_vals[0]) < eps2)
    
    i = 0
    x_prev = x0
    t = t0
    d_prev = None
    grad_prev = None
    prev_flag = False
    beta = 0
    while (i < M):
      grad = get_grad(x_prev)
      if (np.linalg.norm(grad) < eps1):
        return f(x_prev, a, b)
      beta = calc_beta(grad, grad_prev)
      d = calc_dk(grad, beta, d_prev)
      t = calc_tk(x_prev, d)
      x_new = x_prev + t * d
      flag = eps2_check(x_prev, x_new)
      if (flag and prev_flag):
        return f(x_new, a, b)
      prev_flag = flag
      x_prev = x_new
      grad_prev = grad
      d_prev = d
      i += 1
    return f(x_prev, a, b)


In [ ]:
type = int(input())
a,b = map(float,input().split(" "))
if type == 0:
    f = f1
    df = [df1_1,df1_2]
else:
    f = f2
    df = [df2_1,df2_2]
eps1,eps2 = map(float,input().split(" "))
x1,x2 = map(float,input().split(" "))
x = [x1,x2]
print(fletcher_reeves(f,df,a,b,x,10000,0.5,eps1,eps2))

In [ ]:
test1 = [(1, -2, 2, 0.0005, 0.0006, 0.8, 0.6), -7.999999981373431]
test2 = [(1, 1, 2, 0.0008, 0.0004, 0.0, -0.6), 1.0000000617930729]

In [ ]:
run_test(fletcher_reeves, test1)
run_test(fletcher_reeves, test2)

CALCULATED RES: -7.9989217442702865
TEST PASSED!
CALCULATED RES: 1.0004815475315354
TEST PASSED!


# C. Метод Ньютона


---


\

В данной задаче Вам необходимо реализовать метод Ньютона для функции:

$f(x) = (x_0 - x_1^2)^2 + (a-x_0)^2 + b$

\

Максимальная длина шага - 0.5.

В тестовом решении, для поиска оптимального длины шага, использовался поиск Фибоначчи. Вы можете использовать другие методы.

Критерии остановки:
  * Длина градиента в точке меньше $\epsilon_1$

  * Количество итераций меньше 10000 (гарантируется)

  * Две итерации подряд $||x^{k+1} - x^{k}|| < \epsilon_2$ и $||f(x^{k+1}) - f(x^{k})|| < \epsilon_2$ 

\

Можете воспользоваться [шаблоном](https://gist.github.com/evkonovalov/dcd7bea83196ae0ef40195fed284849e)

$a, b$ - коэффициенты

$\epsilon_1, \epsilon_2$ - критерии остановки

$x_0, x_1$ - координаты начальной точки

\

**Формат вывода:**

Значение целевой функции в точке. Проверяется до первого знака после запятой.

In [ ]:
import numpy as np

def f(x,a,b):
    return (x[0]-x[1]**2)**2 + (a-x[0])**2 + b

def df_1(x,a,b):
    return -2*(a-2*x[0]+x[1]**2)

def df_2(x,a,b):
    return -4*x[1]*(x[0]-x[1]**2)

def df_1_1(x,a,b):
    return 4

def df_1_2(x,a,b):
    return -4*x[1]

def df_2_2(x,a,b):
    return -4*(x[0]-3*x[1]**2)

def df_2_1(x,a,b):
    return -4*x[1]

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def ft(t,dk,x,f,a,b):
    x += t * dk
    return f(x,a,b)


def fib_search(f, bounds, tol, coef, max_eps = 0.01):
    n = 0
    fibN = 1
    fibNm1 = 0
    fibNm2 = 0
    l = min(bounds)
    r = max(bounds)
    interv_l = (r - l) / tol
    while fibN < interv_l:
      fibNm2 = fibNm1
      fibNm1 = fibN
      fibN = fibNm1 + fibNm2
      n += 1
    p1 = l + fibNm2/fibN * (r - l)
    p2 = l + fibNm1/fibN * (r - l)
    fp1 = f(p1, *coef)
    fp2 = f(p2, *coef)
    for k in range(1, n-1):
      fibN = fibNm1
      fibNm1 = fibNm2
      fibNm2 = fibN - fibNm1
      if (fp1 > fp2):
        l = p1
        p1 = p2
        p2 = l + (fibNm1/fibN) * (r - l)
        fp1 = fp2
        fp2 = f(p2, *coef) 
      else:
        r = p2
        p2 = p1
        p1 = l + (fibNm2/fibN) * (r - l)
        fp2 = fp1
        fp1 = f(p1, *coef)
    p2 = l + (1/2 - max_eps) * (r - l)
    if fp1 >= f(p2, *coef):
      r = p1
    elif fp1 < f(p2, *coef):
      l = p2
    return (r+l)/2

def newton_method(f, df, H, a, b, x0, M, t0, eps1, eps2):
    def get_H(x_k):
      return np.array([[H[0][0](x_k, a, b), H[0][1](x_k, a, b)], \
                       [H[1][0](x_k, a, b), H[1][1](x_k, a, b)]], dtype='float64')

    def get_dk(x_k, grad):
      cur_H = get_H(x_k)
      if is_pos_def(cur_H):
        return [-np.linalg.inv(cur_H) @ grad.T, True]
      else:
        return [-grad, False]

    def calc_tk(x_k, d_k):
        def f_wrapped(t, x_k, d_k, a, b):
          return f(x_k + t * d_k, a ,b)
        return fib_search(f_wrapped, bounds=[0, 0.5], tol=0.001, coef=[x_k, d_k, a, b])

    def eps2_check(x_0, x_1, f_vals=None):
        if not f_vals:
          f_vals = [f(x_0, a, b), f(x_1, a, b)]
        return (np.linalg.norm(x_1 - x_0) < eps2 and np.linalg.norm(f_vals[1] - f_vals[0]) < eps2)
    H = H
    x_prev = x0
    f_prev = f(x0, a, b)
    t = t0
    x_new = x0
    i = 0
    while (i < M):
      grad = np.array([df[0](x_prev, a, b), df[1](x_prev, a, b)])
      if (np.linalg.norm(grad) < eps1):
        return f_prev
      d, flag = get_dk(x_prev, grad)
      x_new = x_prev + d if (flag) else x_prev + calc_tk(x_prev, d) * d
      f_new = f(x_new, a, b)
      flag = eps2_check(x_prev, x_new, [f_new, f_prev])
      if (flag and prev_flag):
        return f_new
      prev_flag = flag
      f_prev = f_new
      x_prev = x_new
      i += 1
    return f(x_new, a, b)

In [ ]:
a,b = map(float,input().split(" "))
f = f
df = [df_1,df_2]
H = [[df_1_1,df_1_2],[df_2_1,df_2_2]]
eps1,eps2 = map(float,input().split(" "))
x1,x2 = map(float,input().split(" "))
x = [x1,x2]
print(newton_method(f,df,H,a,b,x,10000,0.5,eps1,eps2))

In [ ]:
test1 = [(-3, 2, 0.0009, 0.0005, 0.9, 0.4), 6.500000005600159]
test2 = [(-1, 3, 0.0001, 0.0008, -0.4, 0.2), 3.5000000000002]

In [ ]:
run_test_H(newton_method, test1)
run_test_H(newton_method, test2)

CALCULATED RES: 6.500000007304516
TEST PASSED!
CALCULATED RES: 3.5000000000002
TEST PASSED!


# D. BFGS


---


\

В данной задаче Вам необходимо реализовать метод BFGS для функций двух типов:

$f_0(x) = (x_0 - x_1^2)^2 + (a-x_0)^2 + b$
$f_1(x) = (x_0 - a)^2 + x_0 x_1 + (x_1-b)^2$

\

Максимальная длина шага - 0.5.

В тестовом решении, для поиска оптимального длины шага, использовался поиск Фибоначчи. Вы можете использовать другие методы.

Критерии остановки:
  * Длина градиента в точке меньше $\epsilon$

  * Количество итераций меньше 10000 (гарантируется)

\

Можете воспользоваться [шаблоном](https://gist.github.com/evkonovalov/c472cdfcb30c646e3bd1a33bce69293c)

$t$ - тип функции (0 или 1)

$a, b$ - коэффициенты

$\epsilon_1, \epsilon_2$ - критерии остановки

$x_0, x_1$ - координаты начальной точки

\

**Формат вывода:**

Значение целевой функции в точке. Проверяется до первого знака после запятой.

In [ ]:
import numpy as np

def f1(x,a,b):
    return (x[0]-x[1]**2)**2 + (a-x[0])**2 + b

def f2(x,a,b):
    return (x[0] - a)**2 + x[0]*x[1] + (x[1] - b)**2

def df1_1(x,a,b):
    return -2*(a-2*x[0]+x[1]**2)

def df1_2(x,a,b):
    return -4*x[1]*(x[0]-x[1]**2)

def df2_1(x,a,b):
    return -2*a+2*x[0]+x[1]

def df2_2(x,a,b):
    return -2*b+x[0]+2*x[1]

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def ft(t,dk,x,f,a,b):
    x += t * dk
    return f(x,a,b)

def fib_search(f, bounds, tol, coef, max_eps = 0.01):
    n = 0
    fibN = 1
    fibNm1 = 0
    fibNm2 = 0
    l = min(bounds)
    r = max(bounds)
    interv_l = (r - l) / tol
    while fibN < interv_l:
      fibNm2 = fibNm1
      fibNm1 = fibN
      fibN = fibNm1 + fibNm2
      n += 1
    p1 = l + fibNm2/fibN * (r - l)
    p2 = l + fibNm1/fibN * (r - l)
    fp1 = f(p1, *coef)
    fp2 = f(p2, *coef)
    for k in range(1, n-1):
      fibN = fibNm1
      fibNm1 = fibNm2
      fibNm2 = fibN - fibNm1
      if (fp1 > fp2):
        l = p1
        p1 = p2
        p2 = l + (fibNm1/fibN) * (r - l)
        fp1 = fp2
        fp2 = f(p2, *coef) 
      else:
        r = p2
        p2 = p1
        p1 = l + (fibNm2/fibN) * (r - l)
        fp2 = fp1
        fp1 = f(p1, *coef)
    p2 = l + (1/2 - max_eps) * (r - l)
    if fp1 >= f(p2, *coef):
      r = p1
    elif fp1 < f(p2, *coef):
      l = p2
    return (r+l)/2


def bfgs_method(f,df,a,b, x0, M, t0, eps):
    def get_grad(x_k):
        return np.array([df[0](x_k, a, b), df[1](x_k, a, b)])
    
    def calc_tk(x_k, d_k):
        def f_wrapped(t, x_k, d_k, a, b):
          return f(x_k + t * d_k, a, b)
        return fib_search(f_wrapped, bounds=[0, 0.5], tol=0.001, coef=[x_k, d_k, a, b])

    def calc_Ck(x_1, x_0, C_prev):
        if C_prev is None or x_0 is None:
          return np.eye(2)
        y_k = get_grad(x_1) - get_grad(x_0)
        s_k = x_1 - x_0
        rho_k = 1/np.dot(y_k.T, s_k)
        C_k = np.matmul(np.matmul((np.eye(2, dtype='float64') - rho_k * np.outer(s_k, y_k.T)), C_prev), (np.eye(2, dtype='float64') - rho_k * np.outer(y_k, s_k.T))) + rho_k * np.outer(s_k, s_k.T)
        return C_k
    
    i = 0
    x_prev = None
    x_cur = x0
    C_prev = None
    C = np.eye(2)
    while (i < M):
      grad = get_grad(x_cur)
      if (np.linalg.norm(grad) < eps or np.any(np.isnan(grad))):
        return min(f(x_prev, a, b),f(x_cur, a, b))
      d = np.matmul(-C,  grad)
      t = calc_tk(x_cur, d)
      x_prev = x_cur
      x_cur = x_cur + t * d
      C_prev = C
      C = calc_Ck(x_cur, x_prev, C_prev)
      i += 1
    return f(x_new, a, b)

     
type = int(input())
a,b = map(float,input().split(" "))
if type == 0:
    f = f1
    df = [df1_1,df1_2]
else:
    f = f2
    df = [df2_1,df2_2]
eps = float(input())
x1,x2 = map(float,input().split(" "))
x = [x1,x2]
print(bfgs_method(f,df,a,b,x,10000,0.5,eps))


1
1 1
0.0006
1.0 0.9
0.6666667348886008


In [ ]:
type = int(input())
a,b = map(float,input().split(" "))
if type == 0:
    f = f1
    df = [df1_1,df1_2]
else:
    f = f2
    df = [df2_1,df2_2]
eps = float(input())
x1,x2 = map(float,input().split(" "))
x = [x1,x2]
print(bfgs_method(f,df,a,b,x,10000,0.5,eps))

1
1 1
0.0006
1.0 0.9
0.666666685362031
